In [135]:
from tkinter import *
from setup import *
from random import random
import pandas as pd
import numpy as np
import glob
import os

###################################GLOBALS
CURRENT_SESSION = 1
color="purple"
default_color="white"
ROWS = 8
COLUMNS = 6
NUMJUDGES = 4
#############################################
df = pd.read_csv("gb_id_projects.csv")
# display(df)
pdf = df[['session', 'location','PRESENTATION TYPE','NAME','GUIDEBOOK ID']]
pdf = pdf.loc[pdf['PRESENTATION TYPE']=='poster']

poster_sessions = []

for i in range(4):
    tf = pdf[pdf['session'] == i+1]
    poster_sessions.append(tf.sort_values('location').reset_index(drop=True))
    
# display((poster_sessions[3].loc[poster_sessions[3]['GUIDEBOOK ID']=='8786706'])['location'])

poster_list = poster_sessions[CURRENT_SESSION-1]
print(poster_list)
poster_num =len(poster_list)
print('Num Posters: ' + str(poster_num))
print('Rows: ' + str(ROWS) + ', Cols: ' + str(COLUMNS))
# display(poster_list)

button_list = []
for i in range(ROWS*COLUMNS):
    button_list.append([default_color, NUMJUDGES])

buttons = []   
location_list = []
for i in range(ROWS):
    for j in range(COLUMNS):
        location_list.append(str(i+1)+str(chr(ord('a')+j)))
# print(location_list)
def main():
    window = Tk()
    Grid.rowconfigure(window, 0, weight=1)
    Grid.columnconfigure(window, 0, weight=1)
    frame=Frame(window)
    frame.grid(row=0, column=0, sticky=N+S+E+W)
    for x in range(ROWS):
        Grid.rowconfigure(frame, x, weight=1)
        for y in range(COLUMNS):
            i = x*COLUMNS+y
            Grid.columnconfigure(frame, y, weight=1)
            b = Button(frame,text = str(location_list[x*COLUMNS+y]) + '\n' + 'NEEDS ' + str(NUMJUDGES), bg=default_color, activebackground=default_color)
            b.grid(column=y, row=x)
            b["command"] = lambda b=b, x=x, y=y: click(b, x, y)
            b.grid(row=x, column=y, sticky=N+S+E+W)
            buttons.append(b)
    Grid.columnconfigure(frame, COLUMNS, weight=1)
    Grid.rowconfigure(frame, int(ROWS/2), weight=1)
    b = Button(frame, text = 'update', bg = default_color, activebackground = default_color)
    b["command"] = lambda b=b : update()
    b.grid(column = COLUMNS, row = int(ROWS/2), sticky = N+S+E+W)
    return window

def convert_num(id_list):
    num_list = []
    for x in id_list:
        num_list.append(poster_list.index(x))
    return num_list
    

def click(button,x,y):
    i = x*COLUMNS+y
    num = single_count(i)
#     print(x,y,i)
    if button_list[i][0] is color:
        button_list[i][0] = default_color
        button["bg"] = default_color
        button["activebackground"] = default_color
        button["text"] =str(location_list[x*COLUMNS+y]) + '\n' + 'NEEDS ' + str(NUMJUDGES-num)
    elif button_list[i][0] is default_color:
        button_list[i][0] = color
        button["bg"] = color
        button["activebackground"] = color
        button["text"] = str(location_list[x*COLUMNS+y]) + '\n'+ 'DISPATCHED - ' + str(NUMJUDGES-num)
    if num >= 4:
        button_list[i][0] = "grey"
        button["bg"] = "grey"
        button["activebackground"] = "grey"
        button["text"] = str(location_list[x*COLUMNS+y]) + '\n'+ 'DONE'
def oupdate():
    list_of_files = glob.glob('./inc_guidebook/*') # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    df = pd.read_csv(latest_file, names = ['Judge_ID','Poster_ID','Presenter', 'Question_ID', 'Question', 'Response','Date'])
    ndf = df.iloc[guidebook_loc,:]
    change_list = ndf['Poster_ID']
    change_unique = list(set(change_list))
    change_convert = convert_num(change_unique)
    change_counts = [[x,int(change_list.count(x)/5)] for x in change_unique]

    
    change_list = np.asarray(np.random.choice(poster_num,size=10, replace=False),dtype=np.int)
    change_list= change_list.reshape(10)
#     print(len(buttons))
#     print(random_list)
    for i in change_list:
#         print(i)
        button_column = int(i%COLUMNS)
        button_row = int((i-button_column)/COLUMNS)
#         print(button_row,button_column,i)
        if button_list[i][1] <= 0:
            button_list[i][0] = "grey"
            buttons[i]["bg"] = "grey"
            buttons[i]["activebackground"] = "grey"
            buttons[i]["text"] = str(button_row)+str(button_column)+ '- DONE'
        elif button_list[0] is color:
            button_list[i][0] = default_color
            buttons[i]["bg"] = default_color
            buttons[i]["activebackground"] = default_color
            buttons[i]["text"] = str(x)+str(y)+ '- NEEDS ' + str(button_list[i][1]) + '\n'+str(poster_list[i])
            
        else:
            buttons[i]["text"]=str(button_row)+str(button_column)+ '- NEEDS ' + str(button_list[i][1]) + '\n'+str(poster_list[i])
            button_list[i][0] = default_color
            buttons[i]["bg"] = default_color
            buttons[i]["activebackground"] = default_color

def update():
    list_of_files = glob.glob('./session_update/*') # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    df = pd.read_csv(latest_file, names = ['Judge_ID','Poster_ID','Presenter', 'Question_ID', 'Question', 'Response','Date'])
#     display(df)
    gf = df[df['Poster_ID'].isin(poster_list)]
#     display(gf)
    ndf = df.loc[df['Question']=='Judges Password']
#     ndf = df[df['Response'].str.lower()=='score']
    grouped_df = ndf.groupby('Poster_ID').apply(lambda g: len(g['Judge_ID'].unique()))
#     display(grouped_df)
    display(grouped_df)
    update_list = list(grouped_df.index)
#     print(update_list)
    for project in update_list:
        if int(project) >= 8000000:
            print(project)
            num = grouped_df.loc[project]
            display(poster_list)
            location = poster_list.loc[poster_list['GUIDEBOOK ID']==project]
            print('one')
            display(location)
            location = location['location'].to_string(index=False)
            print('HERE I AM')
            display(location)
            i = location_list.index(location)
            button_column = int(i%COLUMNS)
            button_row = int((i-button_column)/COLUMNS)
    #         print(button_row,button_column,i)
            if num >= 4:
                button_list[i][0] = "grey"
                buttons[i]["bg"] = "grey"
                buttons[i]["activebackground"] = "grey"
                buttons[i]["text"] = str(location_list[COLUMNS*button_row+button_column]) + '\n'+ 'DONE'
            elif button_list[0] is color:
                button_list[i][0] = default_color
                buttons[i]["bg"] = default_color
                buttons[i]["activebackground"] = default_color
                buttons[i]["text"] = str(location_list[COLUMNS*button_row+button_column]) + '\n' + 'NEEDS ' + str(NUMJUDGES-num)

            else:
                buttons[i]["text"]=str(location_list[COLUMNS*button_row+button_column]) + '\n' + 'NEEDS ' + str(NUMJUDGES-num)
                button_list[i][0] = default_color
                buttons[i]["bg"] = default_color
                buttons[i]["activebackground"] = default_color        
#     display(grouped_df)
    
def single_count(i):
    list_of_files = glob.glob('./session_update/*') # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    df = pd.read_csv(latest_file, names = ['Judge_ID','Poster_ID','Presenter', 'Question_ID', 'Question', 'Response','Date'])
    gf = df[df['Poster_ID'].isin(poster_list)]
    display(gf)
    ndf = df.loc[df['Question']=='Judges Password']
#     ndf = df[df['Response'].str.lower()=='score']
#     display(ndf)
    grouped_df = ndf.groupby('Poster_ID').apply(lambda g: len(g['Judge_ID'].unique()))
    item = poster_list.loc[poster_list['location']==location_list[i]]
#     print('HERE IT IS')
#     display(poster_list)
#     print(item['GUIDEBOOK ID'].to_string(index=False))
#     display(grouped_df)
    if item['GUIDEBOOK ID'].to_string(index=False) in list(grouped_df.index):
        return grouped_df.loc[item['GUIDEBOOK ID'].to_string(index=False)]
    else:
        return 0
    
    
w = main()
mainloop()


    session location PRESENTATION TYPE                   NAME GUIDEBOOK ID
0         1       1a            poster     Adekanmbi, Abayomi      8786584
1         1       1b            poster       Hopkins, Melissa      8786592
2         1       1c            poster        Cherif, Yassine      8786588
3         1       1d            poster          Murphy, Caleb      8786848
4         1       1e            poster      Schroers, Melanie      8786860
5         1       2a            poster         Stotz, Miranda      8786599
6         1       2b            poster          Vincze, Haley      8786595
7         1       2c            poster         Payne, Kristen      8786591
8         1       2d            poster         Reid, Campbell      8786841
9         1       2e            poster      Cromer, Alexandra      8786844
10        1       2f            poster         Okolie, Makobi      8786845
11        1       3a            poster         Latin, Tiffany      8786582
12        1       3b     

In [55]:
list_of_files = glob.glob('./session_update/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
df = pd.read_csv(latest_file, names = ['Judge_ID','Poster_ID','Presenter', 'Question_ID', 'Question', 'Response','Date'])
ndf = df.loc[df['Question']=='Judges Password']
grouped_df = ndf.groupby('Poster_ID').apply(lambda g: len(g['Judge_ID'].unique()))
display(grouped_df.loc['7864710'])

2

I think the code below will be pivotal to automatically downloading guidebook data. It should allow me to log in and then get the data on a time interval. 

In [140]:
###################################GLOBALS
CURRENT_SESSION = 1
color="purple"
default_color="white"
ROWS = 8
COLUMNS = 6
NUMJUDGES = 4
#############################################
df = pd.read_csv("gb_id_projects.csv")
# display(df)
pdf = df[['session', 'location','PRESENTATION TYPE','NAME','GUIDEBOOK ID']]
pdf = pdf.loc[pdf['PRESENTATION TYPE']=='poster']

poster_sessions = []

for i in range(4):
    tf = pdf[pdf['session'] == i+1]
    poster_sessions.append(tf.sort_values('location').reset_index(drop=True))
    
# display((poster_sessions[3].loc[poster_sessions[3]['GUIDEBOOK ID']=='8786706'])['location'])

poster_list = poster_sessions[CURRENT_SESSION-1]
print(poster_list)
poster_num =len(poster_list)
print('Num Posters: ' + str(poster_num))
print('Rows: ' + str(ROWS) + ', Cols: ' + str(COLUMNS))
# display(poster_list)

button_list = []
for i in range(ROWS*COLUMNS):
    button_list.append([default_color, NUMJUDGES])

buttons = []   
location_list = []
for i in range(ROWS):
    for j in range(COLUMNS):
        location_list.append(str(i+1)+str(chr(ord('a')+j)))
# print(location_list)

list_of_files = glob.glob('./session_update/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
uf = pd.read_csv(latest_file, names = ['Judge_ID','Poster_ID','Presenter', 'Question_ID', 'Question', 'Response','Date'])
nuf = uf.loc[uf['Poster_ID'].isin(list(poster_list['GUIDEBOOK ID']))]
display(nuf)

    session location PRESENTATION TYPE                   NAME GUIDEBOOK ID
0         1       1a            poster     Adekanmbi, Abayomi      8786584
1         1       1b            poster       Hopkins, Melissa      8786592
2         1       1c            poster        Cherif, Yassine      8786588
3         1       1d            poster          Murphy, Caleb      8786848
4         1       1e            poster      Schroers, Melanie      8786860
5         1       2a            poster         Stotz, Miranda      8786599
6         1       2b            poster          Vincze, Haley      8786595
7         1       2c            poster         Payne, Kristen      8786591
8         1       2d            poster         Reid, Campbell      8786841
9         1       2e            poster      Cromer, Alexandra      8786844
10        1       2f            poster         Okolie, Makobi      8786845
11        1       3a            poster         Latin, Tiffany      8786582
12        1       3b     

,Judge_ID,Poster_ID,Presenter,Question_ID,Question,Response,Date
1,17204376,8786598,"Robinson, Hailey",157036,Poster Content,8: good,November 02 2017 02:07PM
2,17202769,8786649,"Cortez, Raney",157036,Poster Content,8: good,November 02 2017 02:07PM
3,17205224,8786577,"Johnson, Gayland",157036,Poster Content,9: great,November 02 2017 02:07PM
4,17206029,8786591,"Payne, Kristen",157036,Poster Content,8: good,November 02 2017 02:07PM
5,17205477,8786588,"Cherif, Yassine",157036,Poster Content,8: good,November 02 2017 02:06PM
6,17205886,8786614,"Odebiyi, Moyosore",157036,Poster Content,10: exceptional,November 02 2017 02:06PM
7,17205756,8786634,"Brown, Aaliyah",157036,Poster Content,2: poor,November 02 2017 02:06PM
8,17206368,8786841,"Reid, Campbell",157036,Poster Content,6: slightly above average,November 02 2017 02:05PM
9,17205446,8786577,"Johnson, Gayland",157036,Poster Content,8: good,November 02 2017 02:05PM
10,17205477,8786826,"Vasquez, Liliana",157036,Poster Content,5: average,November 02 2017 02:05PM
